In [1]:
'''
!pip install konlpy
!pip install koeda
!pip install matplotlib
!pip install seaborn
'''

'\n!pip install konlpy\n!pip install koeda\n!pip install matplotlib\n!pip install seaborn\n'

In [2]:
# AEDA 사용방법
'''
from koeda import AEDA

aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)
'''

'\nfrom koeda import AEDA\n\naeda = AEDA(\n    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]\n)\n\ntext = "어머니가 집을 나가신다"\n\nresult = aeda(text)\nprint(result)\n'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from koeda import AEDA

In [4]:
train = pd.read_csv("./train.csv")

#### ver1,2 만들 때는 중복데이터 삭제하는 거 까먹음
### 중복 데이터 제거

In [5]:
train_is_duplicated = train.duplicated(subset=["sentence", "subject_entity", "object_entity"], keep=False)
df_duplicated = train.loc[train_is_duplicated, :]
df_duplicated.sort_values(by=["sentence"])
train_drop= train.drop_duplicates(subset=["sentence", "subject_entity", "object_entity"]).reset_index(drop=True)
train_drop

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32418,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32419,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32420,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32421,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


### 각 라벨 별로 문장의 개수 정보 저장

AEDA를 할 때 고려해야할 점:
1. 만약 entity 자리에 문장 기호가 들어간다면 정확도가 낮아지지 않을까?
2. per:place_of_death는 40개, no_relation은 9516개로 편차가 큰데 몇 개를 기준으로 증강해야 할까?

=> 데이터가 정말 부족한 하위 다섯개만 증강

In [6]:
from collections import defaultdict
label_num = defaultdict(int)
idx_1 = [] # "per:place_of_death" # "org:number_of_employees/members"
idx_2 = [] # 'org:dissolved' # 'per:schools_attended'
idx_3 = [] # 'per:religion' # 'org:political/religious_affiliation'
idx_4 = [] # 'per:siblings' # 'per:product'  #'org:founded_by' #'per:place_of_birth'
idx_5 = [] # 'per:other_family' #'per:place_of_residence'

for i in range(32423):
    label_num[train_drop['label'][i]] += 1
    if train_drop['label'][i] == "per:place_of_death" or train_drop['label'][i] == "org:number_of_employees/members":
        # 40개
        idx_1.append(i)
    elif train_drop['label'][i] == 'org:dissolved' or train_drop['label'][i] == 'per:schools_attended':
        # 60~80개
        idx_2.append(i)
    elif train_drop['label'][i] == 'per:religion' or train_drop['label'][i] == 'org:political/religious_affiliation':
        # 90개
        idx_3.append(i)
    elif train_drop['label'][i] == 'per:siblings' or train_drop['label'][i] == 'per:product' or train_drop['label'][i] == 'org:founded_by' or train_drop['label'][i] == 'per:place_of_birth':
        # 130 ~160개
        idx_4.append(i)
    elif train_drop['label'][i] == 'per:other_family' or train_drop['label'][i] == 'per:place_of_residence':
        # 190개
        idx_5.append(i)
    
print(label_num)

defaultdict(<class 'int'>, {'no_relation': 9516, 'org:member_of': 1864, 'org:top_members/employees': 4278, 'org:alternate_names': 1318, 'per:date_of_birth': 1130, 'org:place_of_headquarters': 1194, 'per:employee_of': 3567, 'per:origin': 1234, 'per:title': 2098, 'org:members': 420, 'per:schools_attended': 82, 'per:colleagues': 534, 'per:alternate_names': 997, 'per:spouse': 795, 'org:founded_by': 155, 'org:political/religious_affiliation': 98, 'per:children': 304, 'org:founded': 450, 'org:number_of_employees/members': 48, 'per:place_of_birth': 166, 'org:dissolved': 66, 'per:parents': 518, 'per:religion': 96, 'per:date_of_death': 417, 'per:place_of_residence': 193, 'per:other_family': 190, 'org:product': 380, 'per:siblings': 136, 'per:product': 139, 'per:place_of_death': 40})


In [ ]:
train_1 = train_drop.iloc[idx_1]
aeda = AEDA(
            morpheme_analyzer="Okt",
            punc_ratio=0.3,
            punctuations=[".", ",", "!", "?", ";", ":"],
        )
for i in idx_1+idx_1+idx_1:
    train_1['sentence'][i] = aeda(train_1['sentence'][i])
    #train_1['aeda_sentence'] = train_1['sentence'].apply(lambda x: aeda(x))

#train_1

# *****************************************************************#
train_2 = train_drop.iloc[idx_2]

for i in idx_2+idx_2:
    train_2['sentence'][i] = aeda(train_2['sentence'][i])

#train_2
# *****************************************************************#


C:\Users\2jeon\AppData\Local\Temp\ipykernel_22712\110780002.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['sentence'][i] = aeda(train_1['sentence'][i])


In [ ]:
train_3 = train_drop.iloc[idx_3]

for i in idx_3+idx_3:
    train_3['sentence'][i] = aeda(train_3['sentence'][i])

#train_3
# *****************************************************************#
train_4 = train_drop.iloc[idx_4]

for i in idx_4:
    train_4['sentence'][i] = aeda(train_4['sentence'][i])

#train_4
# *****************************************************************#


In [ ]:
train_5 = train_drop.iloc[idx_5]

for i in idx_5:
    train_5['sentence'][i] = aeda(train_5['sentence'][i])

#train_5

In [ ]:
tmp = pd.concat([train_1, train_2,train_3,train_4,train_5, train_6, train_7])
print(tmp.shape)
# 32423 + 332 = 32755

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax = sns.countplot(y='label', data= tmp, palette = sns.color_palette("pastel"), 
                    order = tmp['label'].value_counts().index, ax=ax)
for p in ax.patches:
    ax.annotate(f'{p.get_width()}', (p.get_x() + p.get_width(), p.get_y() + 0.6), fontsize=10)
plt.title("Relation label countplot")
plt.show()

In [ ]:
result = pd.concat([train_drop, train_1, train_2,train_3,train_4,train_5])
result

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax = sns.countplot(y='label', data=result, palette = sns.color_palette("pastel"), 
                    order = result['label'].value_counts().index, ax=ax)
for p in ax.patches:
    ax.annotate(f'{p.get_width()}', (p.get_x() + p.get_width(), p.get_y() + 0.6), fontsize=10)
plt.title("Relation label countplot")
plt.show()

In [ ]:
result.to_csv('./train_aug_ver4.csv', index=False)

### entity에 문자 들어간 거 필터링 
str.find(subject/object_entity) 리턴값이 -1일 경우 삭제

In [ ]:
i = 1

sentence = result['sentence'][i]
print(sentence)

#{'word': '대성여자중학교', 'start_idx' 
idx1 = result['subject_entity'][i].find('word')
idx2 = result['subject_entity'][i].find('start_idx')
print(sentence[idx+5: start_idx-4])